
# Министерство науки и высшего образования Российской Федерации  
Федеральное государственное бюджетное образовательное учреждение высшего образования  
«Ульяновский государственный технический университет»  
Кафедра «Вычислительная техника»  

**Системы искусственного интеллекта**  
**Лабораторная работа 3**  
«Базы знаний и онтологии»  
**Вариант 23**  

Выполнил  
Студент группы ИВТАСбд-41  
Польгин Илья Андреевич  

Проверил:  
ассистент кафедры «ВТ»  
Хайруллин И.Д.  

Ульяновск, 2025



## Постановка задачи
1. Разработать систему управления доступом с использованием нечеткой логики.  
2. Использовать базу данных SQLite для хранения правил доступа.  
3. Выполнить фаззификацию входных данных (время суток, статус сотрудника, зона доступа).  
4. Реализовать симулятор для проверки корректности работы системы.  
5. Использовать дефаззификацию для получения конкретного уровня доступа.  



## Описание предметной области
Система управления доступом предназначена для автоматизации пропускного режима в организации. Уровень доступа пользователя зависит от:  

- Времени суток (0–23 часа)  
- Статуса сотрудника (посетитель, сотрудник, руководитель, охрана)  
- Зоны доступа (публичная, офис, серверная, секретная)  

Цель системы — обеспечить безопасный и упорядоченный доступ в разные зоны.


In [8]:

import sqlite3
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl


In [9]:

# Входные переменные
time_of_day = ctrl.Antecedent(np.arange(0, 24, 1), 'time_of_day')
employee_status = ctrl.Antecedent([0, 1, 2, 3], 'employee_status')
access_zone = ctrl.Antecedent([0, 1, 2, 3], 'access_zone')

# Выходная переменная
access_permission = ctrl.Consequent(np.arange(0, 101, 1), 'access_permission')

# Нечеткие множества для времени суток
time_of_day['night'] = fuzz.trimf(time_of_day.universe, [0, 0, 6])
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [6, 9, 12])
time_of_day['day'] = fuzz.trimf(time_of_day.universe, [10, 14, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [16, 20, 23])

# Нечеткие множества для статуса сотрудника
employee_status['visitor'] = fuzz.trimf(employee_status.universe, [0, 0, 1])
employee_status['employee'] = fuzz.trimf(employee_status.universe, [0, 1, 2])
employee_status['manager'] = fuzz.trimf(employee_status.universe, [1, 2, 3])
employee_status['security'] = fuzz.trimf(employee_status.universe, [2, 3, 3])

# Нечеткие множества для зоны доступа
access_zone['public'] = fuzz.trimf(access_zone.universe, [0, 0, 1])
access_zone['office'] = fuzz.trimf(access_zone.universe, [0, 1, 2])
access_zone['server'] = fuzz.trimf(access_zone.universe, [1, 2, 3])
access_zone['secret'] = fuzz.trimf(access_zone.universe, [2, 3, 3])

# Нечеткие множества для уровня доступа
access_permission['deny'] = fuzz.trimf(access_permission.universe, [0, 0, 40])
access_permission['limited'] = fuzz.trimf(access_permission.universe, [20, 50, 80])
access_permission['allow'] = fuzz.trimf(access_permission.universe, [60, 100, 100])


In [10]:

def fetch_rules_from_db():
    conn = sqlite3.connect('access_rules.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM rules")
    rows = cursor.fetchall()

    rules = []
    for row in rows:
        time_val, status_val, zone_val, access_val = row[1], row[2], row[3], row[4]
        rule = ctrl.Rule(
            time_of_day[time_val] &
            employee_status[status_val] &
            access_zone[zone_val],
            access_permission[access_val]
        )
        rules.append(rule)

    conn.close()
    return rules


In [11]:

def create_access_control_system():
    rules = fetch_rules_from_db()
    access_ctrl = ctrl.ControlSystem(rules)
    access_sim = ctrl.ControlSystemSimulation(access_ctrl)
    access_permission.defuzzify_method = 'centroid'
    return access_sim


In [12]:

def get_input(prompt, valid_range=None):
    while True:
        try:
            value = int(input(prompt))
            if valid_range and value not in valid_range:
                raise ValueError(f"Ввод должен быть в пределах {valid_range}.")
            return value
        except ValueError as e:
            print(f"Ошибка: {e}")


In [13]:

def simulator():
    print("\nСимулятор системы контроля доступа")
    print("Введите параметры:")

    time_val = get_input("Время (0-23): ", range(0, 24))
    status_val = get_input("Статус (0-визит.,1-сотр.,2-руков.,3-охрана): ", [0, 1, 2, 3])
    zone_val = get_input("Зона (0-общая,1-офис,2-сервер,3-секретная): ", [0, 1, 2, 3])

    access_sim = create_access_control_system()
    access_sim.input['time_of_day'] = time_val
    access_sim.input['employee_status'] = status_val
    access_sim.input['access_zone'] = zone_val
    access_sim.compute()

    result = access_sim.output['access_permission']
    print(f"\nУровень разрешения доступа: {result:.2f}")
    if result < 33:
        print("Доступ: ОТКАЗАН")
    elif result < 66:
        print("Доступ: ОГРАНИЧЕН")
    else:
        print("Доступ: РАЗРЕШЁН")


In [14]:

if __name__ == "__main__":
    while True:
        print("\n1. Запустить симулятор")
        print("2. Выход")
        choice = input("Выберите действие: ")

        if choice == '1':
            simulator()
        elif choice == '2':
            print("Выход.")
            break
        else:
            print("Ошибка ввода.")



1. Запустить симулятор
2. Выход

Симулятор системы контроля доступа
Введите параметры:

Уровень разрешения доступа: 50.00
Доступ: ОГРАНИЧЕН

1. Запустить симулятор
2. Выход
Выход.
